# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
conda activate pydeck-ee

# Install the pydeck-earthengine-layers package from pip
pip install pydeck-earthengine-layers

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

## Authentication

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### Initialize Earth Engine

The above authentication step creates credentials that are stored on your local computer. Those credentials need to be loaded so that Earth Engine and Pydeck will work.

## NOAA Hurricanes example

Now let's make a simple example using Shuttle Radar Topography Mission (SRTM) elevation data. Here we create an `ee.Image` object referencing that dataset.

In [3]:
# Hurricane tracks and points for 2017.
hurricanes = ee.FeatureCollection('NOAA/NHC/HURDAT2/atlantic')

year = '2017'
points = hurricanes.filter(ee.Filter.date(ee.Date(year).getRange('year')))

# Find all of the hurricane ids.
def get_id(point):
    return ee.Feature(point).get('id')
storm_ids = points.toList(1000).map(get_id).distinct()

# Create a line for each hurricane.
def create_line(storm_id):
    pts = points.filter(ee.Filter.eq('id', ee.String(storm_id)))
    pts = pts.sort('system:time_start')
    line = ee.Geometry.LineString(pts.geometry().coordinates())
    feature = ee.Feature(line)
    return feature.set('id', storm_id)

lines = ee.FeatureCollection(storm_ids.map(create_line))

Now we're ready to create the Pydeck layer. The `EarthEngineLayer` makes this simple. Just pass the Earth Engine object to the class.

In [4]:
lines_layer = EarthEngineLayer(
    lines,
    {'color': 'red'},
    id="tracks",
)

In [5]:
points_layer = EarthEngineLayer(
    points,
    {'color': 'black'},
    id="points",
)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [6]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=[points_layer, lines_layer], 
    initial_view_state=view_state
)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…

## Vector

Let's plot these again, but as a vector dataset, instead of a raster dataset

In [7]:
lines_layer = EarthEngineLayer(
    lines,
    as_vector=True,
    get_line_color=[255, 0, 0],
    getLineWidth=1000,
    lineWidthMinPixels=3,
    id="tracks",
    library_url="http://0.0.0.0:8003/pydeck_layer_module.min.js"
)
points_layer = EarthEngineLayer(
    points,
    get_fill_color=[0, 0, 0],
    pointRadiusMinPixels=3,
    getRadius=100,
    getLineColor=[255, 255, 255],
    lineWidthMinPixels=0.5,
    as_vector=True,
    stroked=True,
    id="points",
    library_url="http://0.0.0.0:8003/pydeck_layer_module.min.js"
)

In [8]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=[lines_layer, points_layer], 
    initial_view_state=view_state,
    tooltip=True
)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…

In [12]:
hurricanes.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-75.19999691921689, 35.30000082489263]},
 'id': '000000000000000042fc',
 'properties': {'basin': 'AL',
  'datetime': '2004-08-03T18:00:00',
  'id': 'AL012004',
  'max_wind_kts': 85,
  'min_pressure': 972,
  'name': 'ALEX',
  'numEntries': 25,
  'radii_ne_34kt': 90,
  'radii_ne_50kt': 60,
  'radii_ne_64kt': 30,
  'radii_nw_34kt': 50,
  'radii_nw_50kt': 30,
  'radii_nw_64kt': 15,
  'radii_se_34kt': 90,
  'radii_se_50kt': 50,
  'radii_se_64kt': 30,
  'radii_sw_34kt': 75,
  'radii_sw_50kt': 30,
  'radii_sw_64kt': 20,
  'record_id': '',
  'seq': 1,
  'status': 'HU',
  'system:time_start': 1091556000000,
  'year': 2004}}